# parcel beta stats

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
from nilearn import datasets
from nilearn.image import resample_to_img
import nibabel as nib

In [2]:
# ─── 0) PARAMETERS ─────────────────────────────────────────────────────────
n_rois                     = 200
yeo_networks               = 17
resolution_mm              = 2
STIMULUS_LABEL_SAVE_STRING = "pieman"
deriv_dir                  = "/Volumes/Passport/fmriprep/derivatives"
mr_folder                  = "multi_regression"
multi_dir                  = os.path.join(
    deriv_dir, "RSA_stats", STIMULUS_LABEL_SAVE_STRING, mr_folder
)

In [3]:
# 13 trait labels
TRAIT_LABELS      = [
    "Open-minded","feeling Affectionate","Attentive","Assertive",
    "feeling Gloomy","feeling Peaceful","Agreeable","Judging",
    "feeling Angry","feeling Bewildered","Impulsive",
    "Self-disciplined","Contemplating"
]
TRAIT_SAVE_STRS = [t.replace(" ", "_").replace("-", "_") for t in TRAIT_LABELS]

# Load the BOLD cleaned image
bold_img = nib.load('/Volumes/Passport/fmriprep/derivatives/pieMan_cleaned/sub-002/func/sub-002_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz')

In [4]:
# ─── 1) LOAD ALL SUBJECT‐LEVEL CSVs ────────────────────────────────────────
pattern = os.path.join(multi_dir, "*_multi_parcel_RSA.csv")
all_dfs = []
for fn in sorted(glob.glob(pattern)):
    df = pd.read_csv(fn)           # expects columns: subject, run, parcel, <13 betas>, F_stat
    all_dfs.append(df)
all_df = pd.concat(all_dfs, ignore_index=True)

In [5]:
# ──────────────────────────────────────────────────────────────
# 2) FETCH SCHAEFER ATLAS  ─────────────────────────────────────
# ──────────────────────────────────────────────────────────────

schaefer    = datasets.fetch_atlas_schaefer_2018(
                 n_rois=n_rois,
                 yeo_networks=yeo_networks,
                 resolution_mm=resolution_mm
             )
atlas_img   = nib.load(schaefer['maps'])  # default 2mm MNI - but our images 3x3x4 (Pieman and others) OR 2.5^3 (ie., Black and Forgot)


# Change Schaeffer Labels so 0 is whole brain and 1 corresponds to 1st ROI
labels = schaefer['labels']
# change to string and remove excess
labels = [l.replace(b'17Networks_', b'').decode('utf-8') for l in labels]
# Prepend background label
labels = np.insert(labels, 0, "Background")

In [7]:
# ─── 3) COMPUTE MEAN & T-TEST PER TRAIT × PARCEL ─────────────────────────
results = []
for pid in range(1, n_rois+1):
    parcel_df = all_df[all_df["parcel_num"] == pid]
    row = {
        "parcel_num": pid,
        "parcel_label": labels[pid]
    }
    for trait, sstr in zip(TRAIT_LABELS, TRAIT_SAVE_STRS):
        betas = parcel_df[sstr].dropna()
        mean_beta = betas.mean()
        # one-sample t-test vs. zero
        t_stat, p_val = ttest_1samp(betas, 0.0)
        row[f"mean_beta_{sstr}"] = mean_beta
        row[f"t_{sstr}"] = t_stat
        row[f"p_{sstr}"] = p_val
    results.append(row)

stats_df = pd.DataFrame(results)

In [ ]:
# ─── 4) SAVE OUT THE SUMMARY CSV ─────────────────────────────────────────
out_csv = os.path.join(multi_dir, f"{STIMULUS_LABEL_SAVE_STRING}_parcel_beta_stats_OLS.csv")
stats_df.to_csv(out_csv, index=False)
print("✅ Saved parcel-wise beta statistics →", out_csv)

✅ Saved parcel-wise beta statistics → /Volumes/Passport/fmriprep/derivatives/RSA_stats/pieman/multi_regression/pieman_parcel_beta_stats.csv
